In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.15.0


In [ ]:
from tensorflow.keras.layers import LayerNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15585658829643583175
xla_global_id: -1
]


In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-069mg8pq
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-069mg8pq
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=36556224bd6179403f73db1edc188950b522a3edc81abaab3899cdcf8e653a23
  Stored in directory: /tmp/pip-ephem-wheel-cache-se_2l_q1/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [ ]:
!pip install --upgrade keras
!pip install --upgrade tensorflow
!pip install -U tensorflow
!python -m pip show tensorflow
!pip install --upgrade tensorflow
!python -m pip show tensorflow
!pip install tensorflow-addons
!pip install -q tensorflow-addons
!pip install --upgrade tensorflow
!pip install utils

  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR:

In [ ]:
!pip install pyedflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install padasip

  Preparing metadata (setup.py) ... done
  Created wheel for padasip: filename=padasip-1.2.2-py3-none-any.whl size=51650 sha256=5265067d4dc5c0d8f49a2282da92b4a0ed8d409378313ecac86cb7bb968adff5
  Stored in directory: /root/.cache/pip/wheels/a0/bb/e7/b57f417dbad7d85dbcb177f2c185543ca34a1b36541f115e95
Successfully built padasip


In [ ]:
fileNames = []
fileName_str = []

for i in range(10,11,1):
  name_str = ''
  if(i<10):
    name_str = 'b00'
  else:
    name_str = 'b0'
  fileNames.append(name_str+ str(i)+'.edf')
  fileName_str.append(name_str+ str(i))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks

In [ ]:
import pyedflib
import numpy as np
import pywt  # Importing the Wavelet Transformation module
from scipy.signal import cheby2, filtfilt, butter, find_peaks

class DataUtils:
    def __init__(self, fileNames) -> None:
        super().__init__()
        self.fileNames = fileNames

    def readData(self, sigNum, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        file_name = path + self.fileNames[sigNum]
        f = pyedflib.EdfReader(file_name)
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()
        print("Reading file:", file_name)
        print("Different columns:", signal_labels)
        print("Total number of samples:", f.getNSamples())

        scg = np.zeros((n, f.getNSamples()[0]))
        abdECG = np.zeros((n, f.getNSamples()[0]))

        # Read signals from all available channels
        for i in range(n):
            if i == 3:  # SCG channel
                scg[i, :] = f.readSignal(i)
                scg[i, :] = self.apply_wavelet_transform(scg[i, :])
            elif i == 0:  # ECG channel
                abdECG[i, :] = f.readSignal(i)

        return scg, abdECG

    def apply_wavelet_transform(self, signal):
        # Wavelet decomposition
        coeffs = pywt.wavedec(signal, 'db4', level=4)  # Wavelet decomposition with 'db4' wavelet and 4 decomposition levels
        reconstructed_signal = pywt.waverec(coeffs, 'db4')  # Reconstruction of the signal
        return reconstructed_signal

    def cheby2_bandpass_filter(self, data, lowcut, highcut, fs, order=3, rs=40):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = cheby2(order, rs, [low, high], btype='band')
        y = filtfilt(b, a, data)
        return y

    def qrs_detection(self, ecg, fs):
        qrs_indices = []

        for level_ecg in ecg:
            qrs_level = self.pan_tompkins_on_level(level_ecg, fs)
            qrs_indices.extend(qrs_level)

        qrs_indices = sorted(list(set(qrs_indices)))

        return qrs_indices

    def pan_tompkins_on_level(self, ecg_level, fs):
        nyquist = 0.5 * fs
        low_cutoff = 5
        high_cutoff = 15
        b, a = butter(1, [low_cutoff/nyquist, high_cutoff/nyquist], btype='band')
        ecg_filt = filtfilt(b, a, ecg_level)

        b = np.array([1, 0, -1])
        ecg_diff = np.convolve(ecg_filt, b, mode='same')
        ecg_sq = ecg_diff ** 2

        ma_len = int(0.08 * fs)
        ecg_ma = np.convolve(ecg_sq, np.ones(ma_len)/ma_len, mode='same')

        qrs_idx, _ = find_peaks(ecg_ma, distance=int(0.2 * fs), height=0.2 * np.max(ecg_ma))

        return qrs_idx


fileNames = ['b001.edf','b002.edf','b003.edf','b004.edf','b005.edf','b006.edf','b007.edf','b008.edf','b009.edf','b0010.edf','b0011.edf','b0012.edf','b0013.edf','b0014.edf','b0015.edf','b0016.edf','b0017.edf','b0018.edf','b0019.edf','b0020.edf']  # Replace with your list of file names
data_utils = DataUtils(fileNames)

# Read data from the first file
scg, abdECG = data_utils.readData(0)

# Perform QRS detection on the ECG signals after wavelet transformation
qrs_indices = data_utils.qrs_detection(abdECG, fs=150)


Reading file: /content/drive/MyDrive/drive-download-20230428T081947Z-001/b001.edf
Different columns: ['I', 'II', 'RESP', 'SCG']
Total number of samples: [1365000 1365000 1365000 1365000]


In [ ]:
import pyedflib
import numpy as np
import pywt
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split

class DataUtils:
    def __init__(self, fileNames) -> None:
        super().__init__()
        self.fileNames = fileNames

    def readData(self, sigNum, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        file_name = path + '/' + self.fileNames[sigNum]
        f = pyedflib.EdfReader(file_name)
        n = f.signals_in_file
        signal_labels = f.getSignalLabels()

        scg = np.zeros((n, f.getNSamples()[0]))
        abdECG = np.zeros((n, f.getNSamples()[0]))

        # Read signals from all available channels
        for i in range(n):
            if i == 3:  # SCG channel
                scg[i, :] = f.readSignal(i)
                scg[i, :] = self.apply_wavelet_transform(scg[i, :])  # Apply wavelet transform
            elif i == 0:  # ECG channel
                abdECG[i, :] = f.readSignal(i)

        return scg, abdECG

    def apply_wavelet_transform(self, signal):
        coeffs = pywt.wavedec(signal, 'db4', level=4)  # Wavelet decomposition
        reconstructed_signal = pywt.waverec(coeffs, 'db4')  # Inverse wavelet transform
        return reconstructed_signal



fileNames = ['b001.edf', 'b002.edf', 'b003.edf']  # Replace with your list of file names
dataUtils = DataUtils(fileNames)

# Read SCG signals and apply wavelet transform
scg_signals, _ = dataUtils.readData(0)
print("Original SCG signal shape:", scg_signals.shape)


Original SCG signal shape: (4, 1365000)


Added 2 Discriminators



* line 103, 104 self.image shape c and d
* line 124 - 140 discriminator c and d
* line 151, 152 modified generators
* line 167, 168 added fake_c and d
* line 286 and 287 added batches C and D
* line 299-302 added imgs_c, imgs_d
* line 310, 311 fake_c and d
* line 315, 316 added reconstr_c and d
* line 324 added img_c and d for g_loss
* line 326-332 added dC and dD loss real and fake
* line 335 modified d_loss
* line 376-384 added fake_c and d, reconstr_c and d
* line 394 added imgs_C, fake_D, reconstr_C, imgs_D, fake_C, reconstr_D to gen_imgs


In [ ]:
from sklearn.model_selection import train_test_split

class TrainUtils:
    def __init__(self, fileNames) -> None:
        super().__init__()
        self.fileNames = fileNames
        self.dataUtils = DataUtils(fileNames)

    def prepareData(self, delay=5, path="/content/drive/MyDrive/semester project data set/drive-download-20230428T081947Z-001/b010.edf"):
        scgAll, ecg, fs = self.dataUtils.readData(0, path)
        scgAll = scgAll[range(1), :]
        delayNum = scgAll.shape[0]
        ecgAll = self.dataUtils.createDelayRepetition(ecg, delayNum, delay)

        for i in range(1, len(self.fileNames)):
            scg, ecg = self.dataUtils.readData(i, path)
            scg = scg[range(1), :]
            ecgDelayed = self.dataUtils.createDelayRepetition(ecg, 1, delay)
            scgAll = np.append(scgAll, scg, axis=1)
            ecgAll = np.append(ecgAll, ecgDelayed, axis=1)

        print("ECG all merged shape:: ", ecgAll.shape)

        original_scg = scgAll
        original_ecg = ecgAll
        scgWindows, ecgWindows = self.dataUtils.windowingSig(scgAll, ecgAll, windowSize=1000)
        return scgWindows, ecgWindows

    def trainTestSplit(self, sig, label, trainPercent, shuffle=True):
        print("Splitting into train and test:: ")
        X_train, X_test, y_train, y_test = train_test_split(sig, label, train_size=trainPercent, shuffle=False)
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)
        return X_train, X_test, y_train, y_test


In [ ]:


class WaveletCycleGAN:
    def __init__(self, row, col):
        self.img_rows = row
        self.img_cols = col
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols)
        self.img_shape_b = (self.img_rows, 1)
        self.dataset_name = 'ECG2FECG'

        # Loss weights
        self.lambda_cycle = 4.0  # Cycle-consistency loss
        self.lambda_id = 0.01 * self.lambda_cycle  # Identity loss

        # Build and compile the discriminators
        self.d_A1 = self.build_discriminator(self.img_shape)
        self.d_A2 = self.build_discriminator(self.img_shape_b)
        self.d_B1 = self.build_discriminator(self.img_shape_b)
        self.d_B2 = self.build_discriminator(self.img_shape)

        # Build the generators
        self.g_AB = self.build_generator(self.img_shape)
        self.g_BA = self.build_generator(self.img_shape_b)

        # Optimizer
        optimizer = Adam()

        # Compile discriminators
        self.d_A1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_A2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])

        # Combined model
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape_b)
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[self.d_A1(fake_A), self.d_B1(fake_B),
                                       self.d_A2(fake_B), self.d_B2(fake_A),
                                       fake_B, fake_A,
                                       reconstr_A, reconstr_B])
        self.combined.compile(loss=['huber_loss', 'huber_loss', 'huber_loss', 'huber_loss',
                                     'huber_loss', 'huber_loss', 'huber_loss', 'huber_loss'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id], optimizer=optimizer)

    def build_generator(self, img_shape):
        def conv1DWithSINE(layer_input, filters, f_size=60):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='LeakyReLU')(layer_input)
            d = tfa.layers.InstanceNormalization()(d)
            d = BatchNormalization()(d)
            return d

        def multiply(x):
            mask, image = x
            return image * K.clip(mask, 0.8, 1)

        input = Input(shape=img_shape)
        value = conv1DWithSINE(input, 30, f_size=60)
        att = TransformerBlock(embed_dim=input.shape[1], num_heads=2)(value)
        att = Normalization(axis=1)(att)
        remainedInput = Lambda(multiply)([att, value])
        output_img = conv1DWithSINE(remainedInput, 17, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 1, f_size=1)
        return Model(input, output_img)

    def build_discriminator(self, img_shape):
        def d_layer(layer_input, filters, f_size=33, normalization=True):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='LeakyReLU')(layer_input)
            if normalization:
                d = tfa.layers.InstanceNormalization()(d)
                d = BatchNormalization()(d)
            return d

        img = Input(shape=img_shape)
        d1 = d_layer(img, self.df)
        d2 = d_layer(d1, 13)
        d3 = d_layer(d2, 13)
        d4 = d_layer(d3, 13)
        d5 = d_layer(d4, 13)
        d6 = d_layer(d5, 13)
        validity = d_layer(d6, 1)

        return Model(img, validity)


In [ ]:


class WaveletCycleGAN:
    def __init__(self, row, col):
        self.img_rows = row
        self.img_cols = col
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols)
        self.img_shape_b = (self.img_rows, 1)
        self.dataset_name = 'ECG2FECG'

        # Loss weights
        self.lambda_cycle = 4.0  # Cycle-consistency loss
        self.lambda_id = 0.01 * self.lambda_cycle  # Identity loss

        # Build and compile the discriminators
        self.d_A1 = self.build_discriminator(self.img_shape)
        self.d_A2 = self.build_discriminator(self.img_shape_b)
        self.d_B1 = self.build_discriminator(self.img_shape_b)
        self.d_B2 = self.build_discriminator(self.img_shape)

        # Build the generators
        self.g_AB = self.build_generator(self.img_shape)
        self.g_BA = self.build_generator(self.img_shape_b)

        # Optimizer
        optimizer = Adam()

        # Compile discriminators
        self.d_A1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_A2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])

        # Combined model
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape_b)
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[self.d_A1(fake_A), self.d_B1(fake_B),
                                       self.d_A2(fake_B), self.d_B2(fake_A),
                                       fake_B, fake_A,
                                       reconstr_A, reconstr_B])
        self.combined.compile(loss=['huber_loss', 'huber_loss', 'huber_loss', 'huber_loss',
                                     'huber_loss', 'huber_loss', 'huber_loss', 'huber_loss'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id], optimizer=optimizer)

    def build_generator(self, img_shape):
        def conv1DWithSINE(layer_input, filters, f_size=60):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='relu')(layer_input)
            d = tfa.layers.InstanceNormalization()(d)
            d = BatchNormalization()(d)
            return d

        def multiply(x):
            mask, image = x
            return image * K.clip(mask, 0.8, 1)

        input = Input(shape=img_shape)
        value = conv1DWithSINE(input, 30, f_size=60)
        att = TransformerBlock(embed_dim=input.shape[1], num_heads=2)(value)
        att = Normalization(axis=1)(att)
        remainedInput = Lambda(multiply)([att, value])
        output_img = conv1DWithSINE(remainedInput, 17, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 1, f_size=1)
        return Model(input, output_img)

    def build_discriminator(self, img_shape):
        def d_layer(layer_input, filters, f_size=33, normalization=True):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='relu')(layer_input)
            if normalization:
                d = tfa.layers.InstanceNormalization()(d)
                d = BatchNormalization()(d)
            return d

        img = Input(shape=img_shape)
        d1 = d_layer(img, self.df)
        d2 = d_layer(d1, 13)
        d3 = d_layer(d2, 13)
        d4 = d_layer(d3, 13)
        d5 = d_layer(d4, 13)
        d6 = d_layer(d5, 13)
        validity = d_layer(d6, 1)

        return Model(img, validity)


class TrainUtils:
    def __init__(self):

        pass


from unittest import TestCase
import numpy as np


class TestWaveletCycleGAN(TestCase):

    def __init__(self, methodName: str = ...) -> None:
        super().__init__(methodName)
        self.trainUtils = TrainUtils()

    def test_trainSignal(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        # Adjust data preparation and loading steps to match the requirements of WaveletCycleGAN
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        # Adjust data splitting and reshaping steps accordingly
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        X_train = np.reshape(X_train, [-1, X_train.shape[1], X_train.shape[2]])
        Y_train = np.reshape(Y_train, [-1, Y_train.shape[1], Y_train.shape[2]])

        # Instantiate WaveletCycleGAN instead of CycleGAN
        waveletCycleGAN = WaveletCycleGAN(X_train.shape[1], X_train.shape[2])
        waveletCycleGAN.train(x_train=X_train, y_train=Y_train, epochs=1)

        # Add assertions to validate the correctness of outputs
        # Add more comprehensive testing scenarios as needed

    def divide_test_train(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        # Adjust data preparation and loading steps to match the requirements of WaveletCycleGAN
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        # Adjust data splitting steps accordingly
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        return X_train, X_test, Y_train, y_tes

In [ ]:
from unittest import TestCase


# Adjust imports based on the classes and modules used in WaveletCycleGAN
# from path.to.WaveletCycleGAN import WaveletCycleGAN
# from path.to.TrainUtils import TrainUtils

class TestWaveletCycleGAN(TestCase):

    def __init__(self, methodName: str = ...) -> None:
        super().__init__(methodName)
        self.trainUtils = TrainUtils()

    def test_trainSignal(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        # Adjust data preparation and loading steps to match the requirements of WaveletCycleGAN
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        # Adjust data splitting and reshaping steps accordingly
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        X_train = np.reshape(X_train, [-1, X_train.shape[1], X_train.shape[2]])
        Y_train = np.reshape(Y_train, [-1, Y_train.shape[1], Y_train.shape[2]])

        # Instantiate WaveletCycleGAN instead of CycleGAN
        waveletCycleGAN = WaveletCycleGAN(X_train.shape[1], X_train.shape[2])
        waveletCycleGAN.train(x_train=X_train, y_train=Y_train, epochs=1)

        # Add assertions to validate the correctness of outputs
        # Add more comprehensive testing scenarios as needed

    def divide_test_train(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        # Adjust data preparation and loading steps to match the requirements of WaveletCycleGAN
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        # Adjust data splitting steps accordingly
        X_train, X_test, Y_train, y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        return X_train, X_test, Y_train, y_test


In [ ]:


class TransformerBlock:
    def __init__(self, embed_dim, num_heads):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        # Define other necessary layers and operations for the Transformer block

    def __call__(self, inputs):
        # Implement the forward pass logic for the Transformer block
        x = inputs
        return x


class WaveletCycleGAN:
    def __init__(self, row, col):
        self.img_rows = row
        self.img_cols = col
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols)
        self.img_shape_b = (self.img_rows, 1)
        self.dataset_name = 'ECG2FECG'
        self.df = 64  # Define the df attribute here

        # Loss weights
        self.lambda_cycle = 4.0  # Cycle-consistency loss
        self.lambda_id = 0.01 * self.lambda_cycle  # Identity loss

        # Build and compile the discriminators
        self.d_A1 = self.build_discriminator(self.img_shape)
        self.d_A2 = self.build_discriminator(self.img_shape_b)
        self.d_B1 = self.build_discriminator(self.img_shape_b)
        self.d_B2 = self.build_discriminator(self.img_shape)

        # Build the generators
        self.g_AB = self.build_generator(self.img_shape)
        self.g_BA = self.build_generator(self.img_shape_b)

        # Optimizer
        optimizer = Adam()

        # Compile discriminators
        self.d_A1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_A2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B1.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])
        self.d_B2.compile(loss='MSE', optimizer=optimizer, metrics=['accuracy'])

        # Combined model
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape_b)
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[self.d_A1(fake_A), self.d_B1(fake_B),
                                       self.d_A2(fake_B), self.d_B2(fake_A),
                                       fake_B, fake_A,
                                       reconstr_A, reconstr_B])
        self.combined.compile(loss=['huber_loss', 'huber_loss', 'huber_loss', 'huber_loss',
                                     'huber_loss', 'huber_loss', 'huber_loss', 'huber_loss'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id], optimizer=optimizer)

    def build_generator(self, img_shape):
        def conv1DWithSINE(layer_input, filters, f_size=60):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='LeakyReLU')(layer_input)
            d = tfa.layers.InstanceNormalization()(d)
            d = BatchNormalization()(d)
            return d

        def multiply(x):
            mask, image = x
            return image * K.clip(mask, 0.8, 1)

        input = Input(shape=img_shape)
        value = conv1DWithSINE(input, 30, f_size=60)
        att = TransformerBlock(embed_dim=input.shape[1], num_heads=2)(value)
        att = LayerNormalization(axis=1)(att)
        remainedInput = Lambda(multiply)([att, value])
        output_img = conv1DWithSINE(remainedInput, 17, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 13, f_size=240)
        output_img = conv1DWithSINE(output_img, 1, f_size=1)
        return Model(input, output_img)

    def build_discriminator(self, img_shape):
        def d_layer(layer_input, filters, f_size=33, normalization=True):
            d = Conv1D(filters, kernel_size=f_size, padding='same', activation='LeakyReLU')(layer_input)
            if normalization:
                d = tfa.layers.InstanceNormalization()(d)
                d = BatchNormalization()(d)
            return d

        img = Input(shape=img_shape)
        d1 = d_layer(img, self.df, f_size=64)  # Use self.df with a filter size of 64
        d2 = d_layer(d1, 13)
        d3 = d_layer(d2, 13)
        d4 = d_layer(d3, 13)
        d5 = d_layer(d4, 13)
        d6 = d_layer(d5, 13)
        validity = d_layer(d6, 1)

        return Model(img, validity)

    def train(self, x_train, y_train, epochs):
        # Define the training loop and update the generator and discriminator models
        for epoch in range(epochs):
            # Implement the training logic here
            # You can use the combined model self.combined to train the generator and discriminator models
            # Update the weights of the models based on the training data x_train and y_train


            for batch_x, batch_y in zip(x_train, y_train):
                g_loss, d_loss = self.combined.train_on_batch([batch_x, batch_y], [])
                print(f"Epoch {epoch+1}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")

class TrainUtils:
    def __init__(self):
        pass

    def prepareData(self, delay, path):



        # Simulating data for demonstration purposes
        scgWindows = np.random.rand(100, 1000, 1)
        ecgWindows = np.random.rand(100, 1000, 1)

        return scgWindows, ecgWindows

    def trainTestSplit(self, scgWindows, ecgWindows, split_ratio):

        # Split data into training and testing sets
        num_samples = len(scgWindows)
        split_idx = int(num_samples * split_ratio)

        X_train = scgWindows[:split_idx]
        Y_train = ecgWindows[:split_idx]
        X_test = scgWindows[split_idx:]
        Y_test = ecgWindows[split_idx:]

        return X_train, X_test, Y_train, Y_test

from unittest import TestCase

class TestWaveletCycleGAN(TestCase):
    def __init__(self, methodName: str = ...) -> None:
        super().__init__(methodName)
        self.trainUtils = TrainUtils()

    def test_trainSignal(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        X_train, X_test, Y_train, Y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        Y_train = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1], 1))

        waveletCycleGAN = WaveletCycleGAN(X_train.shape[1], X_train.shape[2])
        waveletCycleGAN.train(x_train=X_train, y_train=Y_train, epochs=1)

        # Add assertions to validate the correctness of outputs
        # Add more comprehensive testing scenarios as needed

    def divide_test_train(self, path="/content/drive/MyDrive/drive-download-20230428T081947Z-001/"):
        scgWindows, ecgWindows = self.trainUtils.prepareData(delay=2, path=path)
        X_train, X_test, Y_train, Y_test = self.trainUtils.trainTestSplit(scgWindows, ecgWindows, 0.75)
        return X_train, X_test, Y_train, Y_test

In [ ]:
!pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from keras.optimizers import Adam
from  keras.callbacks import EarlyStopping

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    n_filters = trial.suggest_categorical('n_filters', [16, 32, 64, 128])

    model = WaveletCycleGAN(n_filters=n_filters)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    scgWindows, ecgWindows = TrainUtils().prepareData(delay=5)
    X_train, X_test, Y_train, Y_test = TrainUtils().trainTestSplit(scgWindows, ecgWindows, 0.75)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) #We have to adjust the value of the patience based on the loss encounterd

    history = model.fit(
        X_train, Y_train,
        epochs=500,
        validation_data=(X_test, Y_test),
        callbacks=[early_stopping],
        verbose=0
    )

    val_loss = np.min(history.history['val_loss'])

    return val_loss


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)
print('Best trial:', study.best_trial.params)

[I 2024-04-25 13:46:56,699] A new study created in memory with name: no-name-54615aa8-f01b-426e-88f0-e1d157a48baa
[W 2024-04-25 13:46:56,702] Trial 0 failed with parameters: {'learning_rate': 0.00012067176476415494, 'n_filters': 16} because of the following error: TypeError("WaveletCycleGAN.__init__() got an unexpected keyword argument 'n_filters'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-21-ed0a05a7a9c7>", line 5, in objective
    model = WaveletCycleGAN(n_filters=n_filters)
TypeError: WaveletCycleGAN.__init__() got an unexpected keyword argument 'n_filters'
[W 2024-04-25 13:46:56,706] Trial 0 failed with value None.


TypeError: WaveletCycleGAN.__init__() got an unexpected keyword argument 'n_filters'

In [ ]:
model = TestWaveletCycleGAN("test_trainSignal")